In [40]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go

In [49]:
# Tutorial source: https://towardsdatascience.com/time-series-forecasting-with-recurrent-neural-networks-74674e289816

In [41]:
df = pd.read_csv('stock_prices.csv', names=['date', 'close'])

In [42]:
df['date'] = pd.to_datetime(df['date'])
df.set_axis(df['date'], inplace=True)

In [43]:
close_data = df['close'].values
close_data = close_data.reshape((-1,1))

In [44]:
split = int(0.8*len(close_data))

In [45]:
close_train = close_data[:split]
close_test = close_data[split:]
date_train = df['date'][:split]
date_test = df['date'][split:]

In [46]:
look_back = 15
train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [47]:
model = Sequential()
model.add(
    LSTM(10, activation='relu', 
         input_shape=(look_back,1))
    )
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/25
40/40 [==============================] - 1s 29ms/step - loss: 20231.9653
Epoch 2/25
40/40 [==============================] - 2s 40ms/step - loss: 17771.8907
Epoch 3/25
40/40 [==============================] - 1s 28ms/step - loss: 14865.3784
Epoch 4/25
40/40 [==============================] - 1s 28ms/step - loss: 22172.3046
Epoch 5/25
40/40 [==============================] - 1s 28ms/step - loss: 25647.5308
Epoch 6/25
40/40 [==============================] - 1s 26ms/step - loss: 24118.9999
Epoch 7/25
40/40 [==============================] - 2s 40ms/step - loss: 22183.2347
Epoch 8/25
40/40 [==============================] - 1s 37ms/step - loss: 20737.4911
Epoch 9/25
40/40 [==============================] - 1s 27ms/step - loss: 18452.2926
Epoch 10/25
40/40 [==============================] - 1s 26ms/step - loss: 19640.1544
Epoch 11/25
40/40 [==============================] - 1s 31ms/step - loss: 16238.2126
Epoch 12/25
40/40 [==============================] - 1s 29ms/step - loss: 

In [48]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()